In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import efficientnet_b0
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import KFold, GroupKFold, train_test_split
from datetime import datetime
from tqdm import tqdm
import albumentations as A
import gc
import matplotlib.pyplot as plt
import math
import multiprocessing
import os
import time
import random
from env import *
from typing import List, Tuple, Union
import pickle
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [87]:
#Kaggle
# DATA_ROOT = "/kaggle/input/hms-harmful-brain-activity-classification/"
# TRAIN_EEG = "train_eegs"
# TRAIN_SPEC = "train_spectrograms"
# TEST_EEG = "test_eegs"
# TEST_SPEC = "test_spectrograms"

#Local
DATA_ROOT = "/home/benluo/HBAC/data/hbac"
TRAIN_EEG = "train_eegs"
TRAIN_SPEC = "train_spectrograms"
TEST_EEG = "test_eegs"
TEST_SPEC = "test_spectrograms"

In [88]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [89]:
train_list = pd.read_csv(os.path.join(DATA_ROOT, "train.csv"))

train_list_reduced = train_list.iloc[:train_list.shape[0]//2,:]

train_list_reduced.to_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

In [90]:
def generate_data(val_folds:int = 5,
                  test_size:float = 0.2,
                  saved_spec:str = "all_spec.pkl",
                  saved_eeg:str = "all_eeg.pkl"):

    train_list = pd.read_csv(os.path.join(DATA_ROOT, "train_reduced.csv"))

    print("All data")
    display(train_list.head())

    # test_df = pd.read_csv(os.path.join(DATA_ROOT, "test.csv"))

    label_cols = train_list.columns[-6:]
    

    #Select first eeg and spectrogram in each spectrogram file
    train_df = train_list[['spectrogram_id','eeg_id','patient_id','spectrogram_label_offset_seconds','eeg_label_offset_seconds']]

    aux = train_list[label_cols].copy()
    
    for label in label_cols:
        train_df[label] = aux[label].values
        
    y_data = train_df[label_cols].values
    y_data = y_data / y_data.sum(axis=1,keepdims=True)
    train_df[label_cols] = y_data

    aux = train_list['expert_consensus'].copy()
    train_df['target'] = aux
    train_df = train_df.reset_index()

    train_df = train_df.sort_values("patient_id")

    test_df = train_df.iloc[int(0.9*train_df.shape[0]):]

    train_df = train_df.iloc[:int(0.9*train_df.shape[0])]

    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    print("Training data")
    display(train_df.head())

    print("Testing data")
    display(test_df.head())


    gkf = GroupKFold(n_splits=val_folds)

    #KFold grouped by patient ID
    for fold, (train_index, val_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
        train_df.loc[val_index, "fold"] = int(fold)
    
    all_eeg = {}
    
    all_spec = {}


    try:

        with open(os.path.join(DATA_ROOT,saved_spec), "rb") as handle:
            all_spec = pickle.load(handle)

        with open(os.path.join(DATA_ROOT,saved_eeg), "rb") as handle:
            all_eeg = pickle.load(handle)

    except:
    
        for idx, row in tqdm(train_list.iterrows()):

            spec_id = row["spectrogram_id"]
            eeg_id = row["eeg_id"]

            if spec_id not in all_spec:

                spec = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_SPEC, str(spec_id)+".parquet"))

                all_spec[spec_id] = spec.iloc[:,1:].values.astype(dtype=np.float16)

            if eeg_id not in all_eeg:

                eeg = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_EEG, str(eeg_id)+".parquet"))

                all_eeg[eeg_id] = eeg.iloc[:,1:].values.astype(dtype=np.float16)
        
        with open(os.path.join(DATA_ROOT, saved_eeg), "wb") as handle:
            pickle.dump(all_eeg, handle)
        
        with open(os.path.join(DATA_ROOT, saved_spec), "wb") as handle:
            pickle.dump(all_spec, handle)
    
    # for i, row in tqdm(test_df.iterrows()):
            
    #     eeg_id = row["eeg_id"]
    #     spec_id = row["spectrogram_id"]

    #     if eeg_id not in test_eeg:

    #         eeg = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_EEG, str(eeg_id)+".parquet"))


    #     if spec_id not in test_spec:
    
    #         spec = pd.read_parquet(os.path.join(DATA_ROOT, TRAIN_SPEC, str(spec_id)+".parquet"))
        
    #     test_eeg[eeg_id] = eeg.iloc[:,1:].values.astype(dtype=np.float16)
    #     test_spec[spec_id] = spec.iloc[:,1:].values.astype(dtype=np.float16)
    
    return train_df, test_df, all_eeg, all_spec, label_cols

In [91]:
train_df, test_df, all_eeg, all_spec, label_cols = generate_data()

All data


,Unnamed: 0,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


Training data


/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/home/benluo/miniconda3/envs/DeepfakeBench/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,41744,41744,802850878,1873660287,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
1,49045,49045,957002006,165634434,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
2,41746,41746,802850878,2057577408,56,290.0,46.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
3,41745,41745,802850878,2057577408,56,244.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other
4,25278,25278,497667405,374550767,56,694.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Other


Testing data


,level_0,index,spectrogram_id,eeg_id,patient_id,spectrogram_label_offset_seconds,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,5410,5410,91996359,3686473557,57272,122.0,122.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
1,5389,5389,91996359,3686473557,57272,72.0,72.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
2,5384,5384,91996359,3686473557,57272,60.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
3,5394,5394,91996359,3686473557,57272,82.0,82.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA
4,5385,5385,91996359,3686473557,57272,62.0,62.0,0.0,0.0,0.0,1.0,0.0,0.0,LRDA


In [92]:
#data processing
class HMSDataset(Dataset):

    def __init__(self,
                 df:pd.DataFrame = None,
                 aug: bool = False) -> None:
        

        super(HMSDataset, self).__init__()

        self.df = df

        #Data augmentation
        self.aug = aug

        self.eeg_sample_freq = 200 # 200 Hz
        self.spec_sample_freq = 0.5 # 0.5 Hz
    
        
        #data augmentation
        self.transforms = A.Compose([
            A.HorizontalFlip(p=0.5)
        ])

    def format_data(self, eeg:np.array, spec:np.array) -> Tuple[torch.tensor]:

        eps = 1e-6

        eeg = eeg.astype(dtype=np.float32)
        spec = spec.astype(dtype=np.float32)

        #Normalizing and getting rid of Nans
        eeg_mean = np.nanmean(eeg.flatten())
        eeg_std = np.nanstd(eeg.flatten())
        eeg = (eeg-eeg_mean)/(eeg_std+eps)
        eeg = np.nan_to_num(eeg, nan=0.0)

        #Normalizing and getting rid of Nans
        spec = np.clip(spec, np.exp(-4), np.exp(8))
        spec = np.log(spec)
        
        spec_mean = np.nanmean(spec.flatten())
        spec_std = np.nanstd(spec.flatten())
        spec = (spec-spec_mean)/(spec_std+eps)
        spec = np.nan_to_num(spec, nan=0.0)
        
        if self.aug:

            eeg = self.transforms(image=eeg)["image"]
            spec = self.transforms(image=spec)["image"]

        eeg = torch.tensor(eeg.copy())
        spec = torch.tensor(spec.copy())

        return eeg, spec


    def __getitem__(self, index) -> dict:
    
        row = self.df.iloc[index]

        eeg_id = row["eeg_id"]
        spec_id = row["spectrogram_id"]

        #EEG Sub-sampling
        start = int(row["eeg_label_offset_seconds"]*self.eeg_sample_freq)
        end = int((row["eeg_label_offset_seconds"]+50)*self.eeg_sample_freq)
        eeg = all_eeg[eeg_id][start:end]
        
        #Spectrogram Sub-sampling
        start = int(row["spectrogram_label_offset_seconds"]*self.spec_sample_freq)
        end = int((row["spectrogram_label_offset_seconds"]+600)*self.spec_sample_freq)
        spec = all_spec[spec_id][start:end]

        # print(all_eeg[eeg_id].shape, eeg.shape, all_spec[spec_id].shape, spec.shape)

        eeg, spec = self.format_data(eeg, spec)

        label = torch.tensor(row[label_cols], dtype=torch.float32)

        return eeg, spec, label


    def __len__(self):

        return self.df.shape[0]

    @staticmethod
    def collate_fn(batch):

        eeg, spec, label = zip(*batch)
        
        if eeg is not None:
            eeg = torch.stack(eeg, dim=0).float().unsqueeze(1)
        
        if spec is not None:
            spec = torch.stack(spec, dim=0).float().unsqueeze(1).expand(-1,3,-1,-1)

        if label is not None:            
            label = torch.stack(label, dim=0)
        
        return {
            "eeg": eeg,
            "spec": spec,
            "label": label
        }
    

In [93]:
class CNNDetector(nn.Module):

    def __init__(self, config={}):

        super(CNNDetector, self).__init__()

        self.backbone_1d = None

        self.backbone_2d = efficientnet_b0(weights="IMAGENET1K_V1")
        
#         self.backbone_2d.conv1 = nn.Conv2d(config.get("in_channels",1), 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
        
        self.backbone_2d.classifier[1] = nn.Linear(1280, config.get("num_classes", 6))
        
#         print(self.backbone_2d)

        self.head = None

    def get_loss(self, pred_dict):

        pass

    def forward(self, data_dict):

        """
        returns a dictionary pred_dict with the logits for loss calculation and gradient descent.
        """
        
#         print(torch.isnan(torch.sum(data_dict["spec"])))

        return self.backbone_2d(data_dict["spec"])

In [94]:
dataset = HMSDataset(df=train_df)
batch_size = 32

print([data.shape for data in dataset[0]])

print(dataset[0])

[torch.Size([10000, 19]), torch.Size([300, 400]), torch.Size([6])]
(tensor([[-1.2819,  0.2882,  0.4570,  ..., -0.2834,  0.6457,  2.7548],
        [-1.5526, -0.0509,  0.1064,  ..., -0.6653,  0.2747,  2.0454],
        [-1.3783,  0.2230,  0.4551,  ..., -0.3216,  0.5962,  2.5634],
        ...,
        [-0.9190,  0.3922,  0.1394,  ..., -0.7086,  0.2932,  0.5750],
        [-1.1416,  0.0568, -0.1324,  ..., -1.0253, -0.0764,  0.1086],
        [-1.1642,  0.0845, -0.0956,  ..., -0.9687, -0.0374,  0.1497]]), tensor([[ 2.9605,  2.9787,  2.9050,  ..., -0.7696, -0.7696, -0.7696],
        [ 2.6904,  2.7197,  2.5592,  ..., -0.8220, -0.8220, -0.8220],
        [ 0.7780,  0.6284,  0.5371,  ..., -0.3574, -0.5286, -0.7696],
        ...,
        [ 3.1196,  3.1458,  2.9791,  ..., -0.7696, -0.7696, -0.7696],
        [ 0.8603,  0.9655,  0.8206,  ..., -0.8220, -0.8220, -0.8220],
        [ 0.5371,  0.5030,  0.5030,  ..., -0.8220, -0.8220, -0.8220]]), tensor([0., 0., 0., 0., 0., 1.]))


In [95]:
#testing

@torch.no_grad()
def validate(model, valid_dataloader):

    model.eval()

    loss_fn = nn.KLDivLoss(reduction="batchmean")

    loss = torch.tensor([0.]).to(device)

    for batch in tqdm(valid_dataloader):

        for key in batch:

            batch[key] = batch[key].to(device)

        predictions = model(batch)
        
        predictions = F.log_softmax(predictions,dim=1)

        loss += loss_fn(predictions,batch["label"])

    loss = loss / len(valid_dataloader)

    model.train()

    print("Validation loss", loss.item())

    return loss.item()

def test(model, test_dataloader):

    model.eval()

    correct = 0

    for batch in test_dataloader:

        for key in batch:
            batch[key] = batch[key].to(device)

        predictions = model(batch)

        predictions = F.softmax(predictions, dim=1)

        predictions = torch.argmax(predictions,dim=1)

        classes = torch.argmax(batch["label"], dim=1)

        correct += torch.sum(predictions == classes, dtype=torch.int)

    model.train()

#training

def train_epoch(model=None,
          train_dataloader=None,
          valid_dataloader=None,
          optimiser = None,
          train_config=None,
          valid_config=None,
          lr_scheduler=None,
          min_valid_loss=100.,
          model_name="model",
          epoch=0,
          logger=None):
    
    """
    Training Function
    """

    assert(train_dataloader is not None)
    
    assert(model is not None)

    #Training Params

    save_model = train_config.get("save_model", True)
    to_model_keys = ["spec", "eeg", "label"]
    valid_step = valid_config.get("valid_step", 1000)
    verbose_step = train_config.get("verbose_step", 10)
    
    
    loss_fn = nn.KLDivLoss(reduction="batchmean")
    
    model.train()
    
    for itr, batch in tqdm(enumerate(train_dataloader)):

#       print(batch["eeg"].shape)

        #Train One Iteration

        #Load data to GPU

        for key in to_model_keys:

            batch[key] = batch[key].to(device)

        predictions = model(batch)
        
        predictions = F.log_softmax(predictions,dim=1)

        loss = loss_fn(predictions,batch["label"])
        
        optimiser.zero_grad()

        loss.backward()

        optimiser.step()
        
        if lr_scheduler is not None:
            lr_scheduler.step()
        
        if itr%verbose_step==0:
            print(f"Training itr {itr}/{len(train_dataloader)}")
            for param_group in optimiser.param_groups:
                lr = param_group['lr']
                break
            print("Training Loss: ", loss, "Learning Rate:", lr)

            logger.log_metric({"KLDiv loss":loss})
        
        #Test
        
        if itr%valid_step==0 and itr>0 and valid_dataloader is not None:
            print("Validation")
            valid_loss = validate(model, valid_dataloader)
            
            if valid_loss < min_valid_loss and save_model:
                min_valid_loss = valid_loss
                save_path = os.path.join(DATA_ROOT,"models")
                os.makedirs(save_path, exist_ok=True)
                torch.save(model.state_dict(), os.path.join(save_path, f"{model_name}_best.pt"))
            logger.log_metric({"Valid loss":valid_loss})
    return min_valid_loss

In [96]:
def train(model=None,
          optimiser=None,
          lr_scheduler=None,
          train_config=None,
          valid_config=None,
          model_name="model",
          logger=None):
    
    train_batch_size = train_config.get("batch_size", 32)
    num_epochs = train_config.get("num_epochs", 10)
    valid_folds = train_config.get("valid_folds", 5)
    valid_batch_size = valid_config.get("batch_size", 32)
    train_workers = train_config.get("workers", 1)
    valid_workers = valid_config.get("workers", 1)

    min_valid_loss = 100

    for epoch in range(num_epochs):

        print("Epoch", epoch)

        #KGFolds
        fold = np.random.randint(valid_folds)
        train_df_fold = train_df[train_df["fold"]!=fold]
        valid_df = train_df[train_df["fold"]==fold]
        train_dataset = HMSDataset(train_df_fold, aug=True)
        valid_dataset = HMSDataset(valid_df, aug=True)

        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size = train_batch_size,
            shuffle=False,
            num_workers=train_workers,
            collate_fn=train_dataset.collate_fn
        )

        valid_dataloader = DataLoader(
            dataset=valid_dataset,
            batch_size = valid_batch_size,
            shuffle=False,
            num_workers=valid_workers,
            collate_fn=valid_dataset.collate_fn
        )

        min_valid_loss = train_epoch(model=model,
                    train_dataloader=train_dataloader,
                    valid_dataloader=valid_dataloader,
                    optimiser=optimiser,
                    train_config=train_config,
                    valid_config=valid_config,
                    lr_scheduler=lr_scheduler,
                    min_valid_loss=min_valid_loss,
                    model_name=model_name,
                    epoch=epoch,
                    logger=logger)
        print("Min Valid Loss", min_valid_loss)

In [98]:
model_config = {}

model = CNNDetector(model_config).to(device)

train_config = {
    "batch_size": 32,
    "num_epochs": 1,
    "lr": 1e-3,
    "valid_folds":5,
    "save_model": True,
    "workers": 1,
    "verbose_step": 100,
    "weight_decay": 0.,
    "valid_folds": 5,
}

valid_config = {
    "batch_size": 16,
    "workers": 1,
    "valid_step": 500
}

lr = train_config.get("lr", 1e-3)
weight_decay = train_config.get("weight_decay", 0.)
model_name = "efficientnetb0"
num_epochs = train_config.get("num_epochs", 1)
folds = train_config.get("valid_folds", 5)
train_batch_size = train_config.get("batch_size", 32)


logger = TensorBoardLogger("logs", name=model_name)

print(num_epochs)

optimiser = torch.optim.Adam(
model.parameters(),
lr=lr,
weight_decay=weight_decay
)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimiser,
    max_lr=lr,
    epochs=num_epochs,
    steps_per_epoch=int((folds-1)/folds*train_df.shape[0]//train_batch_size)
)

torch.cuda.empty_cache()

train(model=model,
      train_config=train_config,
      valid_config=valid_config,
      model_name=model_name,
      optimiser=optimiser,
      lr_scheduler=lr_scheduler,
      logger=logger)

    

1
Epoch 0


0it [00:00, ?it/s]


AttributeError: 'NoneType' object has no attribute 'zero_grad'